In [14]:
import numpy as np
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

class CurrencyLSTMModel:
    def __init__(self, currency_name, data_path, look_back=1):
        self.currency_name = currency_name
        self.data_path = data_path
        self.look_back = look_back
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.model = None

    def load_and_prepare_data(self):
     dataset = read_csv(self.data_path, header=0, index_col=0)
    # 直接用全部数据（只有汇率一列）
     values = dataset.values.astype('float32')
     scaled = self.scaler.fit_transform(values)
     reframed = series_to_supervised(scaled, self.look_back, 1)
     values = reframed.values
     n_train = int(len(values) * 0.7)
     train = values[:n_train, :]
     test = values[n_train:, :]
     train_X, train_y = train[:, :-1], train[:, -1]
     test_X, test_y = test[:, :-1], test[:, -1]
     train_X = train_X.reshape((train_X.shape[0],train_X.shape[1],1))
     test_X = test_X.reshape((test_X.shape[0],test_X.shape[1],1))
     return train_X, train_y, test_X, test_y

    def build_model(self, input_shape):
        model = Sequential()
        model.add(LSTM(6, input_shape=input_shape, return_sequences=True))
        model.add(LSTM(150, return_sequences=True))
        model.add(LSTM(50, return_sequences=False))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='adam')
        self.model = model

    def train(self, epochs=100, batch_size=60):
        train_X, train_y, test_X, test_y = self.load_and_prepare_data()
        self.build_model((train_X.shape[1], train_X.shape[2]))
        history = self.model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size,
                                 validation_data=(test_X, test_y), verbose=2, shuffle=False)
        return history

    def save(self, model_path):
        self.model.save(model_path)

    def load(self, model_path):
        self.model = load_model(model_path)

    def predict(self, X):
        yhat = self.model.predict(X)
        return yhat



In [17]:
import numpy as np
import pandas as pd

# 读取原始数据
dataset = pd.read_csv('Exchange Rate SGP.csv', header=0, index_col=0)
values = dataset.values.astype('float32')

# 归一化
scaled = sgp_model.scaler.transform(values)

# 取最近100天
last_100 = scaled[-100:]  # shape: (100, 1)

# 递归预测未来20天
future_steps = 20
future_preds = []

input_seq = last_100.reshape((1, 100, 1))

for _ in range(future_steps):
    yhat = sgp_model.predict(input_seq)
    future_preds.append(yhat[0, 0])
    # 更新输入序列：去掉最早一天，加上最新预测
    new_input = np.append(input_seq[0, 1:, 0], yhat[0, 0])
    input_seq = new_input.reshape((1, 100, 1))

# 反归一化
future_preds = np.array(future_preds).reshape(-1, 1)
future_real = sgp_model.scaler.inverse_transform(future_preds)
print("未来20天预测汇率：")
print(future_real.flatten())



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
未来20天预测汇率：
[5.3238173 5.3212166 5.320337  5.3209677 5.322647  5.3249025 5.327368
 5.329809  5.332109  5.334228  5.3361716 5.337963  5.3396287 5.3411894
 5.34266   5.3440495 5.345364  5.346607  5.3477826 5.3488927]
